<a href="https://colab.research.google.com/github/cristianopoeta/LEGO/blob/main/Brickset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
txt_separador = '\n'
txt_sets = '''21164
30344
41450
60290
41664'''
 
'''75297
41442
71381
43191
41931
10944
41924
76388
41926
31111
71741
10952
42121
75299
11013
76172
30557
30579
31112
21170
21326
40448
60291''';

In [ ]:
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [ ]:
pdod = pd.options.display
pdod.max_columns = None
pdod.max_colwidth = 50

In [ ]:
def soup_details(txt_set):
    url_base = 'https://brickset.com/sets/'
    url_set = urljoin(url_base, txt_set)
    resp_set = requests.get(url_set)
    resp_set.encoding = resp_set.apparent_encoding
    bs_set = BeautifulSoup(resp_set.text)
    dl_details = bs_set.find(class_='featurebox').find('dl')
    return dl_details
 
def dict_atribs(dl_details):
    dct_atribs = {}
    for dt, dd in zip(dl_details('dt'), dl_details('dd')):
        dct_atribs[dt.text] = dd.text 
    tags_set = [a_tag.text.strip() for a_tag in dl_details.find('dt', string='Tags')
                .find_next_sibling('dd').select('div[id^="tags"] a')]
    if tags_set:
        dct_atribs['Tags'] = tags_set
    else:
        dct_atribs['Tags'] = np.nan
    dt_designer = dl_details.find('dt', string=lambda s: s.startswith('Designer'))
    if dt_designer:
        designers = [a_tag.text.strip() for a_tag 
                     in dt_designer.find_next_sibling('dd').find_all('a')]
        dct_atribs['Designers'] = designers
        try:
            del dct_atribs['Designer']
        except KeyError:
            pass
    else:
        dct_atribs['Designers'] = np.nan
    return dct_atribs

In [ ]:
def frame_sets(dct_sets):
    df_sets = (
        pd.DataFrame.from_dict(dct_sets, orient='index')
        .rename_axis('Set lista')
        .pipe(lambda df:
            df
            .drop(columns=['RRP', 'Price per piece', 'Minifigs'])
            .join(
                [
                    df.RRP
                    .str.extract(r'£([\d\.]+) / \$([\d\.]+) / ([\d\.]+)€')
                    .set_axis(['RRP GBP', 'RRP USD', 'RRP EUR'], axis=1)
                    .astype('float64'),

                    df.Minifigs
                    .str.extract(r'^(\d+)(?: \((\d+) Unique to this set\))?$')
                    .set_axis(['Minifigs', 'Unique to set'], axis=1)
                    .fillna(0)
                    .astype('int64')
                ]
            )
            .assign(**{
                nm_col: df[nm_col].fillna(0).astype('int64')
                for nm_col in ['Year released', 'Pieces']
            })

        )
    )
    return df_sets

In [ ]:
dct_sets = {}
lst_sets = txt_sets.split(txt_separador)
qtd_sets = len(lst_sets)

In [ ]:
for ind_set, txt_set in enumerate(lst_sets, 1):
    print(f'Set {ind_set}/{qtd_sets} [{txt_set}]:', end=' ')
    try:
        if txt_set in dct_sets:
            print('coletado anteriormente. ')
        else:
            print('obtendo atributos...', end=' ')
            dct_sets[txt_set] = dict_atribs(soup_details(txt_set))
            print(f'{len(dct_sets[txt_set])} atributos coletados. ')
    except Exception as e:
        print(e)

Set 1/5 [21164]: obtendo atributos... 19 atributos coletados. 
Set 2/5 [30344]: obtendo atributos... 15 atributos coletados. 
Set 3/5 [41450]: obtendo atributos... 17 atributos coletados. 
Set 4/5 [60290]: obtendo atributos... 21 atributos coletados. 
Set 5/5 [41664]: obtendo atributos... 19 atributos coletados. 


In [ ]:
df_sets = frame_sets(dct_sets)
display(df_sets.head(2))
display(df_sets.shape)

,Set number,Name,Set type,Theme group,Theme,Subtheme,Year released,Tags,Pieces,Packaging,Dimensions,Barcodes,LEGO item numbers,Availability,Rating,Designers,Current value,Notes,Age range,Weight,RRP GBP,RRP USD,RRP EUR,Minifigs,Unique to set
Set lista,,,,,,,,,,,,,,,,,,,,,,,,,
21164,21164-1,The Coral Reef,Normal,Licensed,Minecraft,Minifig-scale,2021,"[Alex, Drowned Zombie, Brick Built Animals, Fi...",92,Box,14.1 x 12.2 x 6.1 cm (5.6 x 4.8 x 2.4 in),EAN: 5702016913569,NA: 6332805EU: 6332804,Retail,✭✭✭✭✭ 4.5 22 ratings,NaN,NaN,NaN,NaN,NaN,8.99,9.99,9.99,2,2
30344,30344-1,Mini Monkey King Warrior Mech,Normal,Action/Adventure,Monkie Kid,Miscellaneous,2021,"[Asia, China, Gift with Purchase, Mech, Micros...",65,Polybag,NaN,NaN,NaN,LEGO exclusive,Not yet rated,[Xiaodong Wen],\nNew: \nUsed:\n,[AU/NZ] Free with qualifying purchases from sh...,NaN,NaN,NaN,NaN,NaN,0,0


(5, 25)

In [ ]:
from datetime import datetime
from google.colab import files
nome_xlsx = f'brickset {datetime.now():%Y-%m-%d %H-%M-%S}.xlsx'
df_sets.to_excel(nome_xlsx, sheet_name='sets', na_rep='', index=True)
files.download(nome_xlsx)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>